In [1]:
from src.data.transforms import Rotate
import PIL.Image as Image
import jax
import jax.numpy as jnp
import numpy as np

#test = jnp.array(np.array(Image.open('tests/data/test.png').convert('RGB')).transpose(2, 1, 0).reshape(1, 3, 224, 224))
test = jnp.array(np.random.randint(0, 255, (32, 3, 224, 224), dtype=np.uint8))
key = jax.random.PRNGKey(0)
s = Rotate((0, 360))

In [1]:
from src.data.loader import DataLoader, DLConfig

config = DLConfig('tests/data', 1)
loader = DataLoader('ssv2', config)

In [ ]:
from tqdm import tqdm

for x in tqdm(loader.train_loader()):
    pass

4it [00:07,  1.99s/it]


KeyboardInterrupt: 

In [2]:
import jax
import haliax as hax

hax.random.randint(jax.random.PRNGKey(0), hax.Axis('t', 10), 0, 255) * 10

NamedArray(array=Array([ 530,  700, 1670,   10, 1220, 2280, 1050, 2110,  820,  570], dtype=int32), axes=(Axis(name='t', size=10),))